In [1]:
import pandas as pd
import fnmatch
import dask.dataframe as dd
from intake.source.utils import reverse_format
import os
import re
import subprocess
from tqdm.auto import tqdm
from pathlib import Path
import shutil

## Create text file containing all files available

In [2]:
def get_file_list(persist_path):
    root = Path("/glade/collections/cmip/CMIP6")
    dirs = [x for x in root.iterdir() if x.is_dir()]
    for directory in tqdm(dirs):
        print(directory)
        stem = directory.stem
        f = open(f"{persist_path}/{stem}.txt", "w")
        cmd = ["find", "-L", directory.as_posix(), "-name", "*.nc"]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=f)
        p.wait()
        

In [3]:
persist_path = "./CMIP6_filelist"
#get_file_list(persist_path)


## Extract attributes of a file using information from CMI6 DRS.


References
 1. CMIP6 DRS: http://goo.gl/v1drZl
 2. Controlled Vocabularies (CVs) for use in CMIP6:
    https://github.com/WCRP-CMIP/CMIP6_CVs
    
    
Directory structure =
```<mip_era>/
    <activity_id>/
        <institution_id>/
            <source_id>/
                <experiment_id>/
                    <member_id>/
                        <table_id>/
                            <variable_id>/
                                <grid_label>/
                                    <version>
```
file name =
```<variable_id>_<table_id>_<source_id>_<experiment_id >_<member_id>_<grid_label>[_<time_range>].nc```
For time-invariant fields, the last segment (time_range) above is omitted.
Example when there is no sub-experiment: `tas_Amon_GFDL-CM4_historical_r1i1p1f1_gn_196001-199912.nc`
Example with a sub-experiment:   `pr_day_CNRM-CM6-1_dcppA-hindcast_s1960-r2i1p1f1_gn_198001-198412.nc`


In [4]:
df = dd.read_csv(f"{persist_path}/*.txt", header=None).compute()
df.columns = ["path"]
df.head()

OSError: ./CMIP6_filelist/*.txt resolved to no files

In [ ]:
len(df)

In [ ]:
def _reverse_filename_format(file_basename, filename_template=None, gridspec_template=None):
    """
    Uses intake's ``reverse_format`` utility to reverse the string method format.
    Given format_string and resolved_string, find arguments
    that would give format_string.format(arguments) == resolved_string
    """
    try:
        return reverse_format(filename_template, file_basename)
    except ValueError:
        try:
            return reverse_format(gridspec_template, file_basename)
        except:
            print(
                f'Failed to parse file: {file_basename} using patterns: {filename_template} and {gridspec_template}'
            )
            return {}
            
def _extract_attr_with_regex(input_str, regex, strip_chars=None):
    pattern = re.compile(regex, re.IGNORECASE)
    match = re.findall(pattern, input_str)
    if match:
        match = max(match, key=len)
        if strip_chars:
            match = match.strip(strip_chars)

        else:
            match = match.strip()

        return match

    else:
        return None
    

exclude_patterns = ['*/files/*', '*/latest/*']
def _filter_func(path):
    return not any(
        fnmatch.fnmatch(path, pat=exclude_pattern) for exclude_pattern in exclude_patterns
    )



In [ ]:
%%time
files = df.path.tolist()
filelist = list(filter(_filter_func, files))

In [ ]:
len(filelist)

In [ ]:
def get_attrs(filepath):
    basename = os.path.basename(filepath)
    dirname = os.path.dirname(filepath)
    filename_template = '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}_{time_range}.nc'

    gridspec_template = (
                '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}.nc'
            )
    
    f = _reverse_filename_format(
            basename, filename_template=filename_template, gridspec_template=gridspec_template
        )

    fileparts = {}
    fileparts.update(f)
    parent = os.path.dirname(filepath).strip('/')
    parent_split = parent.split(f"/{fileparts['source_id']}/")
    part_1 = parent_split[0].strip('/').split('/')
    grid_label = parent.split(f"/{fileparts['variable_id']}/")[1].strip('/').split('/')[0]
    fileparts['grid_label'] = grid_label
    fileparts['activity_id'] = part_1[-2]
    fileparts['institution_id'] = part_1[-1]
    version_regex = r'v\d{4}\d{2}\d{2}|v\d{1}'
    version = _extract_attr_with_regex(parent, regex=version_regex) or 'v0'
    fileparts['version'] = version
    fileparts['path'] = filepath
    return fileparts 

In [ ]:
%%time
entries = list(map(get_attrs, filelist))

In [ ]:
entries[0]

In [ ]:
len(entries)

In [ ]:
df = pd.DataFrame(entries)
df = df.drop_duplicates(subset=['path'], keep='last').reset_index(drop=True)
df.head()

In [ ]:
len(df)

## Separate Decadal Predictions from the rest of activities

- Decadal prediction catalog requires additional columns (`start_year`)

In [ ]:
dcpp = df[df.activity_id=="DCPP"].copy().reset_index(drop=True)
rest = df[~(df.activity_id=="DCPP")].copy().reset_index(drop=True)

In [ ]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "table_id", "variable_id",
           "grid_label", "version", "time_range", "path"]
rest = rest[columns]
rest.head()

In [ ]:
rest.to_csv("/glade/collections/cmip/catalog/glade-cmip6.csv.gz", compression="gzip", index=False)
rest.to_csv("../catalogs/glade-cmip6.csv.gz", compression="gzip", index=False)

In [ ]:
dcpp.head()

In [ ]:
dcpp["start_year"] = dcpp.member_id.map(lambda x: x.split("-")[0][1:])
dcpp["member_id"] = dcpp["member_id"].map(lambda x: x.split("-")[-1])
dcpp.head()

In [ ]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "start_year", "table_id", "variable_id",
           "grid_label", "version", "time_range", "path"]
dcpp = dcpp[columns]
dcpp.head()

In [ ]:
dcpp.to_csv("/glade/collections/cmip/catalog/glade-cmip6-dcpp.csv.gz", compression="gzip", index=False)
dcpp.to_csv("../catalogs/glade-cmip6-dcpp.csv.gz", compression="gzip", index=False)